In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：定制训练表格回归模型进行批量预测并具有可解释性

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_tabular_regression_batch_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_tabular_regression_batch_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_tabular_regression_batch_explain.ipynb">
      在 Google Cloud 笔记本中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex SDK来训练和部署一个用于批量预测的自定义表格回归模型，并提供解释。

### 数据集

本教程使用的数据集是[波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。在本教程中使用的数据集版本已经内置到TensorFlow中。训练好的模型可以预测房屋的中位价格，以1千美元为单位。

### 目标

在本教程中，您将使用Vertex SDK从Python脚本中创建一个具有训练管道的自定义模型，并在Google预构建的Docker容器中执行批量预测并提供解释。您还可以选择使用`gcloud`命令行工具或在线使用Cloud Console来创建自定义模型。

执行的步骤包括：

- 创建一个用于训练模型的Vertex自定义作业。
- 训练TensorFlow模型。
- 检索并加载模型工件。
- 查看模型评估。
- 设置解释参数。
- 将模型上传为Vertex `Model` 资源。
- 进行带解释的批量预测。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud 存储

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud 存储
定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

### 设置本地开发环境

如果您正在使用Colab或Google Cloud Notebooks，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境满足这个笔记本的要求。您需要以下内容：

- 云存储 SDK
- Git
- Python 3
- virtualenv
- 在使用 Python 3 的虚拟环境中运行的 Jupyter 笔记本

[设置 Python 开发环境](https://cloud.google.com/python/setup) 和 [Jupyter安装指南](https://jupyter.org/install) 提供了满足这些要求的详细说明。以下步骤提供了简要的指南：

1. [安装并初始化 SDK](https://cloud.google.com/sdk/docs/)。

2. [安装 Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装 virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用 Python 3 的虚拟环境。激活虚拟环境。

4. 要安装 Jupyter，请在终端 shell 中运行 `pip3 install jupyter`。

5. 要启动 Jupyter，请在终端 shell 中运行 `jupyter notebook`。

6. 在 Jupyter Notebook 仪表板中打开此笔记本。

## 安装

安装最新版本的Python Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

### 重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，以便它能够找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU运行时

本教程不需要GPU运行时。

### 设置您的Google Cloud项目

**无论您的笔记本环境如何，都需要执行以下步骤。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用用于计算/存储成本。

2. [确保为您的项目启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI API、计算引擎API和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行作为shell命令运行，并会将以`$`为前缀的Python变量进行插值。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量在本笔记本的其余部分中使用。以下是Vertex AI支持的区域。我们建议您选择最靠近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不会使用多区域存储桶进行Vertex AI训练。并非所有区域都支持所有Vertex AI服务。

了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能正在使用共享的测试帐户或项目。为了避免在创建资源时用户之间的名称冲突，您为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###验证您的Google Cloud帐户

**如果您正在使用Google Cloud笔记本**，您的环境已经经过身份验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，按照提示进行身份验证以通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**单击创建服务帐户**。

在**服务帐户名称**字段中输入名称，然后单击**创建**。

在**将该服务帐户访问权限授予项目**部分，单击Role下拉列表。在筛选框中键入“Vertex”，然后选择**Vertex管理员**。在筛选框中键入“Storage Object Admin”，然后选择**Storage Object Admin**。

单击创建。一个包含您的密钥的JSON文件将下载到您的本地环境。

在下面的单元格中键入您服务帐户密钥的路径作为GOOGLE_APPLICATION_CREDENTIALS变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您的笔记本环境如何，都需要执行以下步骤。**

当您为 Python 初始化 Vertex SDK 时，您需要指定一个云存储暂存桶。该暂存桶是您的数据集和模型资源在会话期间保留的位置。

请在下面设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

## 为 Python 初始化 Vertex SDK

为你的项目和对应的存储桶初始化 Python 的 Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量 `TRAIN_GPU/TRAIN_NGPU` 和 `DEPLOY_GPU/DEPLOY_NGPU`，以使用支持 GPU 的容器映像和分配给虚拟机实例（VM）的 GPU 数量。例如，要使用一个 GPU 容器映像，分配给每个 VM 的是 4 个 Nvidia Telsa K80 GPU，您应该指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，请指定 `(None, None)`来使用一个在 CPU 上运行的容器映像。

在您的区域了解更多关于硬件加速器支持的信息 [这里](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)

*注意*：TF 2.3 之前的 GPU 支持版本会在这个教程中加载自定义模型时出现失败。这是一个已知问题，在 TF 2.3 中已经修复--这是由生成的静态图操作导致的。如果您在自定义模型中遇到此问题，请使用支持 GPU 的 TF 2.3 容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 设置预构建的容器

设置用于训练和预测的预构建的Docker容器镜像。

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`以配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个虚拟CPU 3.75GB内存。
     - `n1-highmem`：每个虚拟CPU 6.5GB内存。
     - `n1-highcpu`：每个虚拟CPU 0.9GB内存。
 - `vCPUs`：[\2, 4, 8, 16, 32, 64, 96 \]内CPU核心数

*注意：以下配置不支持用于训练：*

 - `standard`：2个虚拟CPU
 - `highcpu`：2、4和8个虚拟CPU

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在您已经准备好开始为波士顿房屋创建自定义模型和训练。

### 检查培训包

#### 包布局

在开始培训之前，您将查看如何为自定义培训任务组装 Python 包。解压后，该包包含以下的目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到 Docker 镜像的操作环境中的说明。

文件 `trainer/task.py` 是执行自定义培训任务的 Python 脚本。*注意*，当我们在工作池规范中引用它时，我们用点(`trainer.task`)替换目录斜杠并省略文件后缀(`.py`)。

#### 包组装

在以下单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 内容

在下一个单元格中，您将编写训练脚本 task.py 的内容。我不会详细说明，只是供您浏览。总结如下：

- 从命令行中获取要保存模型工件的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 获取。
- 从 TF.Keras 内置数据集中加载波士顿房屋数据集。
- 使用 TF.Keras 模型 API 构建一个简单的深度神经网络模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分发策略。
- 训练模型（`fit()`）并指定 epochs 为 `args.epochs`。
- 将训练好的模型保存到指定的模型目录（`save(args.model_dir)`）。
- 将每个特征的最大值保存到指定的参数文件中（`f.write(str(params))`）。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))


def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)


# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
BATCH_SIZE = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_dnn_model()

# Train the model
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=GLOBAL_BATCH_SIZE)
model.save(args.model_dir)

请将培训脚本存储在您的云存储桶中

接下来，将培训文件夹打包成压缩的tar文件，并将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_boston.tar.gz

### 创建和运行自定义训练任务

要训练一个自定义模型，您需要完成两个步骤：1）创建一个自定义训练任务，2）运行这个任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建一个自定义训练任务，包括以下参数：

- `display_name`：自定义训练任务的可读名称。
- `container_uri`：训练容器镜像。
- `requirements`：训练容器镜像的包要求（例如，pandas）。
- `script_path`：训练脚本的相对路径。

In [ ]:
job = aip.CustomTrainingJob(
    display_name="boston_" + TIMESTAMP,
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs==0.7.1", "tensorflow-datasets==4.4"],
)

print(job)

### 准备你的命令行参数

现在为你的自定义训练容器定义命令行参数：

- `args`: 传递给作为容器入口点设置的可执行文件的命令行参数。
  - `--model-dir`: 在我们的演示中，我们使用这个命令行参数来指定存储模型工件的位置。
      - 直接: 将云存储位置作为命令行参数传递给你的训练脚本（设置变量 `DIRECT = True`），
      - 间接: 服务将云存储位置作为环境变量 `AIP_MODEL_DIR` 传递给你的训练脚本（设置变量 `DIRECT = False`）。在这种情况下，你需要在作业规范中告诉服务模型工件的位置。
  - `"--epochs=" + EPOCHS`: 训练的时期数。
  - `"--steps=" + STEPS`: 每个时期的步数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_NAME, TIMESTAMP)

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

#### 运行自定义训练作业

接下来，您可以运行自定义作业来启动训练作业，通过调用方法 `run` 并传入以下参数：

- `args`：传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1 表示单节点训练）。
- `machine_type`：用于计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要连接到工作人员副本的加速器数量。
- `base_output_dir`：用于将模型工件写入的云存储位置。
- `sync`：是否阻塞直到作业完成。

In [ ]:
if TRAIN_GPU:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=True,
    )
else:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

model_path_to_deploy = MODEL_DIR

## 加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud Storage 存储桶中。现在从 Cloud Storage 存储桶加载它，然后您就可以执行一些操作，比如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras 中的 `model.load_model()` 方法，将保存模型的 Cloud Storage 路径传递给它，路径由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

local_model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在让我们看看模型的表现如何。

### 加载评估数据

您将从`tf.keras.datasets`中加载波士顿住房测试（留出）数据，使用`load_data()`方法。这将返回一个包含两个元素的元组数据集。第一个元素是训练数据，第二个是测试数据。每个元素也是一个包含两个元素的元组：特征数据和相应的标签（自住房屋的中位价值）。

您不需要训练数据，因此我们将其加载为`(_, _)`。

在您对数据进行评估之前，您需要对其进行预处理：

`x_test`：
1. 通过将每个值除以该列的最大值来对每列数据进行归一化（重新缩放）。这将每个单个值替换为0到1之间的32位浮点数。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import boston_housing

(_, _), (x_test, y_test) = boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)


def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float32)
    return feature


# Let's save one data item that has not been scaled
x_test_notscaled = x_test[0:1].copy()

for _ in range(13):
    x_test[_] = scale(x_test[_])
x_test = x_test.astype(np.float32)

print(x_test.shape, x_test.dtype, y_test.shape)
print("scaled", x_test[0])
print("unscaled", x_test_notscaled)

### 执行模型评估

现在评估一下自定义任务中的模型表现如何。

In [ ]:
local_model.evaluate(x_test, y_test)

获取Serving函数签名

您可以重新加载模型到内存中，并查询每个层对应的签名，从而获得模型的输入和输出层的签名。

在进行预测请求时，您需要将请求发送到Serving函数而不是模型，因此您需要知道Serving函数的输入层名称 - 这将在稍后进行预测请求时使用。

您还需要知道Serving函数的输入和输出层名称，以构建解释元数据 - 这将在随后讨论。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)
serving_output = list(loaded.signatures["serving_default"].structured_outputs.keys())[0]
print("Serving function output:", serving_output)

input_name = local_model.input.name
print("Model input name:", input_name)
output_name = local_model.output.name
print("Model output name:", output_name)

### 解释规范

要在进行预测时获得解释，您必须启用解释功能，并在将自定义模型上传到 Vertex `Model` 资源时设置相应的设置。这些设置被称为解释元数据，包括：

- `parameters`：这是用于在模型上进行解释的可解释性算法的规范。您可以在以下选项之间进行选择：
  - Shapley - *注意*，不推荐用于图像数据 -- 可能需要很长时间才能完成
  - XRAI
  - 综合梯度（Integrated Gradients）
- `metadata`：这是用于将算法应用于您的自定义模型的规范。

#### 解释参数

让我们首先更深入地了解解释算法的设置。

#### Shapley

为每个特征分配结果的功劳，并考虑各种特征的不同排列。该方法提供了精确 Shapley 值的采样近似。

用例：
  - 表格数据的分类和回归。

参数：

- `path_count`：这是算法处理的特征上路径数。准确估计 Shapley 值需要 M! 条路径，其中 M 是特征数量。对于 CIFAR10 数据集，这将是 784 (28*28)。

对于任何非常重要的特征数量，这太耗时了。您可以将特征上的路径数减少到 M * `path_count`。

#### 综合梯度（Integrated Gradients）

一种基于梯度的方法，可以高效地计算具有与 Shapley 值相同公理属性的特征归因。

用例：
  - 表格数据的分类和回归。
  - 图像数据的分类。

参数：

- `step_count`：这是近似余下总和所需的步数。步数越多，积分近似越准确。一般的经验法则是 50 步，但步数越多，计算时间也越长。

#### XRAI

基于综合梯度方法，XRAI 评估图像的重叠区域，创建一个突出显示相关图像区域而不是像素的显著性地图。

用例：
  - 图像数据的分类。

参数：

- `step_count`：这是近似余下总和所需的步数。步数越多，积分近似越准确。一般的经验法则是 50 步，但步数越多，计算时间也越长。

在下一个代码单元格中，将变量 `XAI` 设置为您要在自定义模型上使用的解释算法。

In [ ]:
XAI = "ig"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aip.explain.ExplanationParameters(PARAMETERS)

#### 说明元数据

让我们首先深入了解说明元数据，它包括：

- `outputs`: 输出量中的一个标量值，用于表示要解释的内容。例如，在分类问题中的概率输出\[0.1, 0.2, 0.7\]中，我们希望解释0.7的值。考虑下面的公式，其中输出值为 `y`，这就是我们要解释的内容。

    y = f(x)

考虑下面的公式，其中输出值为 `y` 和 `z`。由于我们只能对一个标量值进行解释，我们必须选择是否要解释输出值 `y` 或 `z`。在这个例子中假设模型是目标检测，y和z分别表示边界框和物体分类。你需要选择要解释这两个输出中的哪一个。

    y, z = f(x)

`outputs` 的字典格式如下：

    { "outputs": { "[your_display_name]":
                   "output_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]: 你为要解释输出值分配的人类可读名称。一个常见的例子是 "概率"。<br/>
 -  "output_tensor_name": 用于识别要解释输出层的键/值字段。<br/>
 -  [layer]: 要解释的输出层。在单一任务模型中，比如表格回归器，它是模型中的最后（最顶部）一层。
</blockquote>

- `inputs`: 要求对输出做出贡献的特征。考虑下面的公式，其中 `a` 和 `b` 是特征。我们必须选择要解释如何对输出做出贡献的特征。假设这个模型部署用于 A/B 测试，其中 `a` 是预测的数据项，`b` 用于识别模型实例是 A 还是 B。你需要选择 `a`（或某些子集）作为特征，而不是 `b`，因为它不会对预测做出贡献。

    y = f(a,b)

`inputs` 的最小字典格式如下：

    { "inputs": { "[your_display_name]":
                  "input_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]: 你为要解释输入值分配的人类可读名称。一个常见的例子是 "特征"。<br/>
 -  "input_tensor_name": 用于识别特征贡献的输入层的键/值字段。<br/>
 -  [layer]: 用于特征贡献的输入层。在单一输入张量模型中，它是模型中的第一个（最底部）一层。
</blockquote>

由于模型的输入是表格形式的，你可以指定以下两个额外字段作为报告/可视化的辅助：

<blockquote>
 - "encoding": "BAG_OF_FEATURES" : 表示输入是一组表格特征。<br/>
 - "index_feature_mapping": [特征名称] : 每个特征的人类可读名称列表。在这个例子中，我们使用数据集中指定的特征名称。<br/>
 - "modality": "numeric": 表示字段值为数字。
</blockquote>

In [ ]:
INPUT_METADATA = {
    "input_tensor_name": serving_input,
    "encoding": "BAG_OF_FEATURES",
    "modality": "numeric",
    "index_feature_mapping": [
        "crim",
        "zn",
        "indus",
        "chas",
        "nox",
        "rm",
        "age",
        "dis",
        "rad",
        "tax",
        "ptratio",
        "b",
        "lstat",
    ],
}

OUTPUT_METADATA = {"output_tensor_name": serving_output}

input_metadata = aip.explain.ExplanationMetadata.InputMetadata(INPUT_METADATA)
output_metadata = aip.explain.ExplanationMetadata.OutputMetadata(OUTPUT_METADATA)

metadata = aip.explain.ExplanationMetadata(
    inputs={"features": input_metadata}, outputs={"medv": output_metadata}
)

## 上传模型

接下来，使用`Model.upload()`方法将您的模型上传到一个`Model`资源，使用以下参数：

- `display_name`：`Model`资源的可读名称。
- `artifact`：训练模型工件的云存储位置。
- `serving_container_image_uri`：提供服务的容器镜像。
- `sync`：是否异步或同步执行上传。
- `explanation_parameters`：配置`Model`预测解释的参数。
- `explanation_metadata`：描述解释`Model`输入和输出的元数据。

如果`upload()`方法是异步运行的，您可以随后使用`wait()`方法阻塞直到完成。

In [ ]:
model = aip.Model.upload(
    display_name="boston_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    explanation_parameters=parameters,
    explanation_metadata=metadata,
    sync=False,
)

model.wait()

发送批量预测请求

发送一个批量预测请求到你的部署模型。

制作测试条目

您将使用合成数据作为测试数据项。不要担心我们使用的是合成数据 - 我们只是想展示如何进行预测。

### 制作批量输入文件

现在制作批量输入文件，您将把它存储在您的本地云存储桶中。与图像、视频和文本不同，表格的批量输入文件仅支持 CSV 格式。对于 CSV 文件，您需要制作：

- 第一行是包含特征（字段）名称的标题行。
- 每个剩余的行是一个单独的预测请求，包含相应的特征值。

例如：

    "feature_1", "feature_2". ...
    value_1, value_2, ...

In [ ]:
! gsutil cat $IMPORT_FILE | head -n 1 > tmp.csv
! gsutil cat $IMPORT_FILE | tail -n 10 >> tmp.csv

! cut -d, -f1-16 tmp.csv > batch.csv

gcs_input_uri = BUCKET_NAME + "/test.csv"

! gsutil cp batch.csv $gcs_input_uri

### 发起批量解释请求

现在您的 Model 资源已经训练完成，您可以通过调用 batch_predict() 方法来进行批量预测，以下是参数：

- `job_display_name`: 用于批量预测作业的可读名称。
- `gcs_source`: 一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`: 用于存储批量预测结果的 Cloud 存储位置。
- `instances_format`: 输入实例的格式，可选'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`: 输出预测结果的格式，可选'csv'或'jsonl'。默认为'jsonl'。
- `generate_explanations`: 设置为 `True` 以生成解释信息。
- `sync`: 如果设置为 True，调用将在等待异步批量作业完成时阻塞。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

batch_predict_job = model.batch_predict(
    job_display_name="boston_" + TIMESTAMP,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_NAME,
    instances_format="csv",
    predictions_format="jsonl",
    machine_type=DEPLOY_COMPUTE,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    generate_explanation=True,
    sync=False,
)

print(batch_predict_job)

### 等待批量预测作业完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批量预测作业完成。

In [ ]:
if not os.environ["IS_TESTING"]:
    batch_predict_job.wait()

获取解释

接下来，从已完成的批量预测作业中获取解释结果。

结果被写入了您在批量预测请求中指定的Cloud Storage输出存储桶。您可以调用iter_outputs()方法获取生成结果的每个Cloud Storage文件的列表。每个文件都以CSV格式包含一个或多个解释请求：

- CSV标题 + 预测标签
- CSV行 + 解释，每个预测请求一个

In [ ]:
if not os.environ["IS_TESTING"]:
    import tensorflow as tf

    bp_iter_outputs = batch_predict_job.iter_outputs()

    explanation_results = list()
    for blob in bp_iter_outputs:
        if blob.name.split("/")[-1].startswith("explanation"):
            explanation_results.append(blob.name)

    tags = list()
    for explanation_result in explanation_results:
        gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{explanation_result}"
        with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
            for line in gfile.readlines():
                print(line)

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 管道
- 模型
- 终端
- AutoML训练作业
- 批处理作业
- 自定义作业
- 超参数调整作业
- Cloud存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME